# Adding Features 

### Introduction

In this lesson, we'll add a couple of features to our project and review how the codebase fits together along the way.  Let's get started. 

### Initial Steps

Begin by cloning the repository located [here](https://github.com/jigsawlabs-student/foursquare-flask-api).  And then checkout the branch `add_cli`.  Our first large task will be to boot up the application, but simply performing that task will involve some work.

* As an initial step, install packages listed in the `requirements.txt` file.

### Setting up the database

Then let's set up our database.  

* Create a database called `foursquare_practice`. 
    * Do so from the command line with the `-c` flag.
* Then run the migrations in the `db/migrations/create_tables.sql` file.

Now that our database and related tables are created, let's load in some data.  We have a number of CSV files in the `/data` folder.  We can copy in the data into our tables with a command in the following format:

```SQL
COPY table_name
FROM 'absolute/path/to/data.csv' 
DELIMITER ',' 
CSV HEADER;
```

Load in each csv file in the data folder to the related table.

> **Note**: Begin with the least dependent tables (eg. states then cities)

We can check that at least some of the data was loaded properly.  For example, confirm that there are 134 records in the venues table.

### Getting Started



Now the first step will be to see if we can start up the website.  To do so, we'll have to set some of the environmental variables so that we can connect to the correct database.  Create a `.env` file, and add the following variables assigned to the appropriate values.

```shell
# .env
DB_PASSWORD = 
DB_NAME = 
DB_HOST = 
DB_USER = 
DEBUG = 
TESTING = 
```

Then try booting up the backend by running the flask application with the `manage.py` file.  

>**Note:** If unsure how to boot up flask with `manage.py`, run `python3 manage.py` and look at the output for the correct approach.

Then find the route that displays a list of restaurants, and then navigate to that route.  

> The individual restaurants may be different, but we should see something like the following.

> <img src="./venues_json.png" width="60%">

Ok, now it's time to get the frontend setup.

* First, install the packages listed in the `requirements.txt` file for the frontend.
* Then bootup the streamlit application

It should look something like the following.

> <img src="./streamlit_frontend.png" width="60%">

### Reading the Code

Before adding additional features to the codebase, let's make sure we understand how this first graph is being displayed.  To do so, we can work backwards.  

The plotly graph is displayed with the following code in the `index.py` file.

> <img src="./venue_plot.png" width="60%">

We can see that the scatter plot calls the functions `venue_names`, and `venue_ratings`, with each function taking an argument of `venues`.  And now let's see where the variable `venues` comes from.

We can see it defined a few lines above.

> <img src="./venues_code.png" width='80%'>

So we see that venues comes from the `find_venues` code, and takes in a value of price, which is defined from the dashboard.  And we can learn more about `find_venues` by looking at the file it's imported from, `view_functions`.  There, we'll see the following:

```python
# view_functions.py

import requests
API_URL = "http://127.0.0.1:5000/venues/search"

def find_venues(price):
    response = requests.get(API_URL, params = {'price': price})
    return response.json()
```

So this is the line of code that hits our api, via the `API_URL`, and then passes these venues to the `venue_names` and `venue_ratings` functions to return a list of values.  
```python
# names
['chipotle', 'cafe mogador', 'numero uno']
# ratings
[3, 5, 4.5]
```

Which are then displayed in the plot that we see on the screen.

> Notice our approach for reading the code.  We started with the code directly related to our view, and then traced it back from the functions of `venue_names`,  `venue_ratings` and the variable `venues`, to see how this plot was created.

### Updating the View

Now let's add a new plot that graphs the number of likes.  We want a plot that displays the `venue_names` along the x-axis and the number of likes of each venue along the y-axis.  Define any functions in the `view_functions.py` file.

> Note: We'll have to make sure that we only display those venues that have a `likes` value -- so both the venue names and venue likes may need to be filtered to meet this requirement.

<img src="./venue_likes.png" width="60%">

### Updating the API

Now that we have updated a plot on the frontend, let's see if we can update the API.  Currently the API does not display each venue's `category`.  Update the API so that, if there is a category this value is returned.  When complete the API should look like the following:

> So notice that the first dictionary does not have a category listed, as none was available.  But the second entry does.

> <img src="./category_name.png" width="40%">

> **Hint**:  To solve this, use the same technique of starting with the top most layer, and working backwards to see where to update the code.  This time, the top most layer is the `/venues` route.

> Bonus: If you would like to make use of this code, update the venue ratings plot with the following code:

```python
st.header('Venues')
st.write('Venue ratings')
# update here
colors = category_colors(venues)
scatter = go.Scatter(x = venue_names(venues, True), y = venue_ratings(venues, True), marker = {'color': colors }, mode = 'markers')
fig = go.Figure(scatter)
st.plotly_chart(fig)
```

### Adding an Aggregate Function

Now before moving on, let's think about how our graphs above have worked.  We have plotted attributes - like the rating or number of likes of each *restaurant venue*.  We get this by making a request to the `/venues` url, which returns a list of venue dictionaries.

Now this time, let's develop a plot that displays the name of a `category` and the corresponding average rating of restaurants in each category.  When it's done, it will look like the following:

> <img src="./category_plot.png" width="60%">

To develop this, we'll need to query our database to send back data from our api to our frontend.  And the data will need to be in a differerent format in each layer.

Let's see the different formats:

* database: return a list of records 
    * `[(1, 'mexican', 4.3), (2, 'chinese', 4.7)]`
* api: return a list of dictionaries (json)
    * `[{'id': 1, 'cuisine': 'mexican', 'rating': 4.3}, {'id': 2, 'cuisine': 'chinese', 'rating': 4.7}]`
* plot: display x values and y values
  * `go.Scatter(x = ['mexican', 'chinese'], y = [4.3, 4.7])`

So each layer of our application returns data in the proper format - the database returns a list of records, the api returns a list of dictionaries, and the plotly graph takes in data of a list of x values and y values.

### Adding the Function

Now let's build this out.  We'll begin with the database method that returns the list of categories and the related average rating.

* Before writing out this method, we need to determine the correct location for it.  Which model does this method belong in?

* Is this an instance method or a class method.  (We'll have this method return multiple average ratings.)

> Ok, so method belongs in the `Category` class, and should be a class method (as we are performing the calculation over the collection of categories and not a single category).

Now define the following class method.

* `Venue.avg_ratings()`

> First just return a list of the average ratings, and then as an almost fun bonus, return the categories from highest to lowest rating, remove the categories with null values, and round the ratings to two digits.

And then run `console.py` and check that the function works.

<img src="./avg_ratings_console.png" width="100%">

* Returning a list of dictionaries

Now we successfully made the database call, but to return this data from the API to our frontend as JSON, we always want to return a list of dictionaries.  Normally, we would take this data and create objects out of it, and then from there create our dictionaries.  But here, it seems like it would be easier to simply create a dictionary of for each category that looks like the following.

```python
[{'name': 'Sandwich Place', 'rating': 9.3}, {'name': 'Vegetarian', 'rating': 8.9}]
```

So update the `avg_ratings` function so that it returns the data in this format.

> This time, when we call `Category.avg_ratings`

```python
Category.avg_ratings()
# [{'name': 'Sandwich Place', 'rating': Decimal('9.30')}, {'name': 'Vegetarian / Vegan Restaurant', 'rating': Decimal('8.90')}, {'name': 'Indian Restaurant', 'rating': Decimal('8.90')}, {'name': 'Taco Place', 'rating': Decimal('8.04')}
```

* Returning from the api

Ok, now that we have a way to return our list of dictionaries, it's time to have this data returned from our API.

If it's working properly, we should be able to make a request to the categories end point, and see a list of dictionaries with the name of the category and corresponding average rating.

> <img src="./categories_api.png" width="40%">

### Plotting our Data

Ok, now that our API returns data in the correct format, it's time to plot the data.  We'll want a plot of the names of the categories and the related average rating.  The plot should look like the following:

* Write a function called `find_categories`, that takes no arguments and returns a list of categories as the list of dictionaries from our API.

* Then create the plot of categories and corresponding ratings, writing any additional functions along the way.

> <img src="./avg_categories_plot.png" width="60%">

### Summary

In this lesson, we worked through adding features to our codebase.  We saw that to do so, a large step was in first understanding the different components of the codebase, and then making the appropriate fix.  We also saw that a good technique to determine what code to update was to trace our code from the outer most layer, and then see what code is relied upon.